# BT01: Thu thập dữ liệu

Họ tên: **Nguyễn Ngọc Băng Tâm**

MSSV: **1712747**

---
## Cách làm bài và nộp bài
&#9889; Bạn lưu ý là mình sẽ dùng chương trình hỗ trợ chấm bài nên bạn cần phải tuân thủ chính xác qui định mà mình đặt ra, nếu không rõ thì hỏi, chứ không nên tự tiện làm theo ý của cá nhân.

**Cách làm bài**

Bạn sẽ làm trực tiếp trên file notebook này. Đầu tiên, bạn điền họ tên và MSSV vào phần đầu file ở bên trên. Trong file, bạn làm bài ở những chỗ có ghi là:
```python
# YOUR CODE HERE
raise NotImplementedError()
```
hoặc đối với những phần code không bắt buộc thì là:
```python
# YOUR CODE HERE (OPTION)
```
hoặc đối với markdown cell thì là:
```markdown
YOUR ANSWER HERE
```
Tất nhiên, khi làm thì bạn xóa dòng `raise NotImplementedError()` đi.
Đối những phần yêu cầu code thì thường ở ngay phía dưới sẽ có một (hoặc một số) cell chứa các bộ test để giúp bạn biết đã code đúng hay chưa; nếu chạy cell này không có lỗi gì thì có nghĩa là qua được các bộ test. Trong một số trường hợp, các bộ test có thể sẽ không đầy đủ; nghĩa là, nếu không qua được test thì là code sai, nhưng nếu qua được test thì chưa chắc đã đúng.

Trong khi làm bài, bạn có thể cho in ra màn hình, tạo thêm các cell để test. Nhưng khi nộp bài thì bạn xóa các cell mà bạn tự tạo, xóa hoặc comment các câu lệnh in ra màn hình. Bạn lưu ý <font color=red>không được tự tiện xóa các cell hay sửa code của Thầy</font> (trừ những chỗ được phép sửa như đã nói ở trên).

Trong khi làm bài, thường xuyên `Ctrl + S` để lưu lại bài làm của bạn, tránh mất mát thông tin.


*Nên nhớ mục tiêu chính ở đây là <font color=green>học, học một cách chân thật</font>. Bạn có thể thảo luận ý tưởng với bạn khác, nhưng <font color=green>code và bài làm phải là của bạn, dựa trên sự hiểu thật sự của bạn</font>. <font color=red>Nếu vi phạm thì sẽ bị 0 điểm cho toàn bộ môn học.</font>*

**Cách nộp bài**

Khi chấm bài, đầu tiên mình sẽ chọn `Kernel` - `Restart Kernel & Run All Cells`, để restart và chạy tất cả các cell trong notebook của bạn; do đó, trước khi nộp bài, bạn nên chạy thử `Kernel` - `Restart Kernel & Run All Cells` để đảm bảo mọi chuyện diễn ra đúng như mong đợi.

Sau đó, bạn tạo thư mục nộp bài theo cấu trúc sau:
- Thư mục `MSSV` (vd, nếu bạn có MSSV là 1234567 thì bạn đặt tên thư mục là `1234567`)
    - Thư mục `BT01`
        - File `BT01-ThuThapDuLieu.ipynb` (không cần nộp các file khác)

Cuối cùng, bạn nén thư mục `MSSV` này lại và nộp ở link trên moodle. <font color=red>Bạn lưu ý tuân thủ chính xác cấu trúc này.</font>

---
## Import

In [1]:
from requests_html import HTMLSession
import requests
import json
import time # Dùng để sleep chương trình
import pandas as pd # Dùng để đọc và hiển thị file csv (Pandas sẽ được học chi tiết ở buổi tới)
import datetime as dt # Dùng để xử lý dữ liệu thời gian
import re
# YOUR CODE HERE (OPTION) 
# Nếu cần các thư viện khác thì bạn có thể import ở đây
import urllib.robotparser # Kiểm tra file robot.txt có được phép crawl không

---
## 1. Thu thập dữ liệu từ web bằng cách parse HTML (5đ)
[Coursera](https://coursera.org) là trang cung cấp các khóa học online, là một nguồn tài liệu hữu ích để học tập. Trong phần này, bạn sẽ thu thập dữ liệu về các khóa học Data Science ở Coursera. 

Cụ thể là bạn sẽ viết hàm `collect_courses` ở bên dưới. Hàm này có các input như sau:
- `course_urls_file`: là chuỗi, cho biết tên của file txt chứa các đường link đến các khóa học của Coursera (tất cả các khóa học, chứ không phải chỉ các khóa học về Data Science), mỗi đường link nằm trên một dòng; bạn sẽ đi theo đường link để lấy thông tin của khóa học. File này đã được cung cấp cho bạn: file "course_urls.txt" (mình tạo ra file này bằng cách parse [trang XML này](https://www.coursera.org/sitemap~www~courses.xml)). Theo file [robots.txt](https://www.coursera.org/robots.txt) thì các link trong file này đều được phép thu thập (bạn kiểm tra lại xem có đúng không). 
- `key_words`: là list, cho biết các từ khóa để lọc ra các link cần lấy. Ví dụ: `key_words=['data-science', 'datascience']` sẽ chỉ thu thập thông tin từ các link mà có từ "data-science" HOẶC "datascience" (chẳng hạn: https://www.coursera.org/learn/spatial-data-science).
- `sleep_time`: là số thực, cho biết sau mỗi lần gửi request và parse lấy dữ liệu thì sẽ sleep bao nhiêu giây trước khi gửi tiếp lần request kế (sleep như vậy để tránh gửi quá nhiều request lên Coursera trong một thời gian ngắn). Trong hàm `collect_courses` bên dưới mình để giá trị mặc định là 1 giây. Trong Python, bạn sleep chương trình bằng câu lệnh `time.sleep(số giây)`.
- `courses_file`: là chuỗi, cho biết tên của file csv dùng để lưu dữ liệu thu thập được; đây là file output (file này không có sẵn mà sẽ được tạo ra sau khi chạy hàm của bạn). Mình có cung cấp file output đúng "correct_courses.csv" (khi gọi hàm `collect_courses` với `key_words=['data-science', 'datascience']`) để bạn hình dung được nội dung cần có của file output. File này có các phần tử trên mỗi dòng được phân tách nhau bởi tab (`\t`), và file này mã hóa utf-8. Bạn có thể xem file csv này ở JupyterLab bằng cách double-click vào file ở phía bên trái JupyterLab rồi chọn "Delimiter" là tab; hoặc xem ở dạng text bằng cách chuột phải vào file rồi "Open With", "Editor". File này gồm có các cột:
    - `url`: đường link đến khóa học.
    - `title`: tựa đề của khóa học.
    - `provider`: trường cung cấp khóa học.
    - `rating`: điểm số của khóa học (tại thời điểm bạn thu thập dữ liệu thì giá trị của cột này có thể sẽ hơi khác so với file output đúng của mình).
    - `num_ratings`: số lượng người đã cho điểm cho khóa học (tại thời điểm bạn thu thập dữ liệu thì giá trị của cột này có thể sẽ hơi khác so với file output đúng của mình).

In [2]:
def collect_courses(course_urls_file, key_words, courses_file, sleep_time=1):
    # YOUR CODE HERE
    
    rp = urllib.robotparser.RobotFileParser()
    rp.set_url('https://www.coursera.org/robots.txt')
    rp.read()
    
    courses = []
    
    with open(course_urls_file, 'r') as inp:
        for line in inp:
            # Check robots.txt file
            if rp.can_fetch('*', line):
                 # Extract urls with key_words
                result = [line for key_word in key_words if re.search(key_word, line)]
                courses.extend(result)
            else:
                print(f"Can't fetch {line}")
   
    # Remove unnecessary characters in urls
    urls = [course.replace('\n', '') for course in courses]    

    titles, providers, ratings, num_ratings = [], [], [], []
    
    session = HTMLSession()
        
    for url in urls:
        # sleep before request a session 
        time.sleep(sleep_time)
        r = session.get(url)
        
        if r.html.find('.banner-title', first=True) is not None:
            title = r.html.find('.banner-title', first=True).text
            
            raw_provider = r.html.find('.rc-Partner__title')
            
            raw_provider = ([each_provider.text for each_provider in raw_provider])
            provider = ' '.join(raw_provider)

            XDPRating = r.html.find('.XDPRating', first=True)
            
            rating = XDPRating.find('.number-rating-expertise', first=True).text
            rating = rating.replace('\nstars', '')
            num_rating = XDPRating.find('.ratings-count-expertise-style', first=True).text
            num_rating = num_rating.replace('ratings', '')
        else:
            title = provider = rating = num_rating = ''

        titles.append(title)
        providers.append(provider)
        ratings.append(rating)
        num_ratings.append(num_rating)
    
#     # Create a DataFrame using 5 lists
    courses_list = [urls, titles, providers, ratings, num_ratings]
    df = pd.DataFrame(courses_list).transpose()
    df.columns = ['url', 'title', 'provider', 'rating', 'num_ratings']       
    df.to_csv(courses_file, sep='\t', index=False, header=True, encoding='utf-8')

In [3]:
# TEST
collect_courses('course_urls.txt', ['data-science', 'datascience'], 'courses.csv')
courses = pd.read_csv('courses.csv', sep='\t')
assert courses.shape == (26, 5)

In [4]:
# TEST
courses

,url,title,provider,rating,num_ratings
0,https://www.coursera.org/learn/code-free-data-...,Code Free Data Science,University of California San Diego,4.4,56
1,https://www.coursera.org/learn/introduction-cl...,Introduction to Clinical Data Science,University of Colorado System,4.7,246
2,https://www.coursera.org/learn/what-is-datasci...,What is Data Science?,IBM,4.7,"40,660"
3,https://www.coursera.org/learn/the-data-scienc...,The Data Science of Health Informatics,Johns Hopkins University,4.7,117
4,https://www.coursera.org/learn/spatial-data-sc...,Spatial Data Science and Applications,Yonsei University,4.4,325
5,https://www.coursera.org/learn/data-science-me...,Data Science Methodology,IBM,4.6,"14,989"
6,https://www.coursera.org/learn/datasciencemath...,Data Science Math Skills,Duke University,4.5,"8,044"
7,https://www.coursera.org/learn/sql-data-science,Databases and SQL for Data Science,IBM,4.7,"12,132"
8,https://www.coursera.org/learn/data-science-fo...,Data Science for Business Innovation,EIT Digital Politecnico di Milano,4.2,88
9,https://www.coursera.org/learn/python-for-data...,Python для анализа данных,Moscow Institute of Physics and Technology ФРО...,4.5,85


---
## 2. Thu thập dữ liệu từ web bằng cách dùng web API (5đ)
Trong phần này, bạn sẽ dùng GitHub API để thu thập các thùng chứa về Data Science. Để đơn giản, bạn sẽ dùng API không chứng thực (không cần đăng ký tài khoản, không cần key) của GitHub. 

### 2.1. Thu thập ít hơn hoặc bằng 1000 thùng chứa (2.5đ)
Đầu tiên, bạn sẽ viết hàm `collect_repositories` ở bên dưới. Hàm này có các input:
- `start_date` và `end_date`: là chuỗi, cho biết là muốn tìm các thùng chứa được tạo từ ngày nào đến ngày nào (bao gồm cả 2 đầu). Ví dụ, `start_date='2017-01-01'` (năm-tháng-ngày) và `end_date='2017-01-31'`.
- `keyword`: là chuỗi, cho biết là muốn tìm kiếm các thùng chứa với từ khóa nào (xuất hiện ở tên thùng chứa hoặc phần mô tả của thùng chứa). Ví dụ, `keyword='data science'`.
- `per_page`: là số nguyên, cho biết là muốn bao nhiêu kết quả trên một page. Hiện giờ GitHub cho tối đa là `per_page=100`.

Trong hàm này bạn sẽ vào đường link `f'https://api.github.com/search/repositories?q={keyword} created:{start_date}..{end_date}&per_page={per_page}'` (dạng f-string trong Python) để thu thập dữ liệu. Ví dụ, bạn có thể vào thử ở web browser: https://api.github.com/search/repositories?q=data%20science%20created:2017-01-01..2017-01-31&per_page=100 (khi nhập một đường link vào web browser và enter thì một số ký tự sẽ bị mã hóa; ví dụ khoảng trắng sẽ bị mã hóa là %20 hoặc +), hoặc dùng thư viện Requests:

In [5]:
# Thư viện requests sẽ tự động mã hóa khoảng trắng cho bạn
# Bạn có thể xem đường link sau khi đã mã hóa bằng câu lệnh: r.url
r = requests.get('https://api.github.com/search/repositories?q=data science created:2017-01-01..2017-01-31&per_page=100&page=1')

In [6]:
# Xem 100 ký tự đầu tiên trong chuỗi JSON lấy được
print(r.text[:100])

{"total_count":1570,"incomplete_results":false,"items":[{"id":79030157,"node_id":"MDEwOlJlcG9zaXRvcn


Bạn thấy "total_count" có giá trị là 1615, nghĩa là tìm được 1615 thùng chứa. Tuy nhiên, đây chỉ là trang đầu tiên và chỉ có 100 thùng chứa (số lượng phần tử của "items"). Link tới trang kế tiếp nằm trong headers của nội dung gửi về từ server.

In [7]:
r.headers['Link']

'<https://api.github.com/search/repositories?q=data+science+created%3A2017-01-01..2017-01-31&per_page=100&page=2>; rel="next", <https://api.github.com/search/repositories?q=data+science+created%3A2017-01-01..2017-01-31&per_page=100&page=10>; rel="last"'

Link tới trang kế tiếp là link mà ở bên cạnh phía phải có từ `rel="next"`. `rel="last"` nghĩa là trang cuối. Vì đây là trang đầu tiên nên chỉ có "next" và "last", trang ở giữa còn có thêm "first" (trang đầu) và "prev" (trang trước). Bạn có thể xem thêm ở [document](https://developer.github.com/v3/guides/traversing-with-pagination/).

Nếu để ý thì bạn thấy trang cuối là `page=10`, nghĩa là tất cả các trang chỉ có $10\times100=1000$ thùng chứa. Trong khi đó, "total_count" là 1615. Lý do: "the GitHub Search API provides up to **1,000 results for each search**" ([document](https://developer.github.com/v3/search/)). Trong hàm này chỉ yêu cầu bạn lấy hết kết quả ở tất cả các trang là được ($\le1000$ kết quả). Việc lấy $>1000$ kết quả sẽ làm ở phần kế tiếp.

Một yêu cầu nữa trong hàm này là khi hết số lượng request được phép trong 1 phút (với API không chứng thực thì hiện nay được 10 request / phút) thì bạn phải cho chương trình đợi cho đến khi được reset lại (thường thì không cần đợi tới 1 phút vì thời gian 1 phút là tính từ lần request đầu tiên). Cách làm là bạn sẽ cho chương trình sleep 1 giây rồi dậy xem được reset chưa, nếu chưa thì lại sleep 1 giây rồi dậy ... Bạn lưu ý là, <font color=red>không được giả định lần request đầu tiên là chắc chắn thành công</font> (vì khi chấm bài mình sẽ phải chạy nhiều bài và có thể đến bài của bạn đã hết lượt request được phép). Để xem số lượng request còn lại, bạn có thể xem header:

In [8]:
r.headers['X-RateLimit-Remaining'] # Lưu ý kết quả trả về là chuỗi

'9'

Khi số lượng request còn lại bằng 0 thì có 2 trường hợp có thể xảy ra:
- Lần request vừa xong thành công (là lần request cuối cùng được phép), lúc này `r.ok` bằng `True`.
- Lần request vừa xong không thành công (đã hết lượng request được phép), lúc này `r.ok` bằng `False`.

Output trả về của hàm `collect_repositories`:
- `repositories`: là list các thùng chứa lấy được (list gồm "item" của tất cả các trang).

In [9]:
def collect_repositories(start_date, end_date, keyword='data science', per_page=100):
    repositories = []
    # YOUR CODE HERE
    page = 1  
    
    # the last page is always page 10
    while page <= 10:
        url = f'https://api.github.com/search/repositories?q={keyword} created:{start_date}..{end_date}&per_page={per_page}&page={page}'
        r = requests.get(url)
            
        if r.ok == True:
            json_pydata = r.json()
            if len(json_pydata['items']) == 0: # no items left on that page
                break
            repositories.extend(json_pydata['items'])    
    
            page += 1
        else:
            time.sleep(1)
            
    
    return repositories 

In [10]:
# TEST
start_date = '2017-01-01'
end_date = '2017-01-15'
keyword = 'data science'
per_page = 100
repositories = collect_repositories(start_date, end_date, keyword, per_page)
print(len(repositories))

url = f'https://api.github.com/search/repositories?q={keyword} created:{start_date}..{end_date}'
finish = False
while finish == False:
    r = requests.get(url)
    if r.ok == True:
        json_pydata = r.json()
        if json_pydata['incomplete_results'] == False: 
            total_count = json_pydata['total_count']
            finish = True
    else:
        time.sleep(1)
assert len(repositories) == total_count 

666


### 2.2. Thu thập nhiều hơn 1000 thùng chứa (2.5đ)
Để thu thập hết 1615 thùng chứa trong ví dụ ở trên, ta sẽ: 
- Chia khoảng thời gian các thùng chứa được tạo mà ta cần tìm kiếm ra các khoảng thời gian nhỏ hơn. Ví dụ chia 2017-01-01..2017-01-15 thành 2017-01-01..2017-01-07, 2017-01-08..2017-01-14, 2017-01-15..2017-01-15; ở đây, mỗi khoảng thời gian nhỏ gồm 7 ngày (kể cả 2 đầu), khoảng thời gian nhỏ sau cùng có thể sẽ không đủ 7 ngày (do bị lẻ). Trong Python, để làm việc với dữ liệu thời gian, bạn có thể dùng thư viện `datetime`; bạn có thể tự tìm hiểu về thư viện này ở: [document](https://docs.python.org/3.7/library/datetime.html), [Corey's video](https://www.youtube.com/watch?v=eirjjyP2qcQ), Google.
- Tìm kiếm với khoảng thời gian nhỏ hơn này bằng hàm `collect_repositories` ở trên (với khoảng thời gian đủ nhỏ sẽ có $\le1000$ thùng chứa).
- Kết hợp kết quả từ các lần tìm kiếm với các khoảng thời gian nhỏ này lại.

Bạn sẽ cài đặt các bước này ở hàm `collect_all_repositories` dưới đây. Hàm này có các input:
- `start_date`, `end_date`, `keyword`, `per_page`: các input này giống như ở hàm `collect_repositories`.
- `step`: là số nguyên, cho biết khoảng thời gian nhỏ gồm bao nhiêu ngày (tính cả 2 đầu). Trong ví dụ ở trên, `step=7`.

Output trả về của hàm này:
- `all_repositories`: là list tất cả các thùng chứa ("item") lấy được.

In [11]:
def collect_all_repositories(start_date, end_date, step, keyword='data science', per_page=100):
    all_repositories = []
    # YOUR CODE HERE

    # Create (start_date, end_date) intervals
    date_intervals = [] 
    
    # Calculate the time difference when binning the date
    dt_end_date = dt.datetime.strptime(end_date, "%Y-%m-%d")
    dt_start_date = dt.datetime.strptime(start_date, "%Y-%m-%d")
    time_left = dt_end_date - dt_start_date
  
    while (time_left.days > step): 
        dt_next_date = dt_start_date + dt.timedelta(days=step - 1)
        next_date = dt.datetime.strftime(dt_next_date, "%Y-%m-%d")
        start_date = dt.datetime.strftime(dt_start_date, "%Y-%m-%d")
        date_intervals.append((start_date, next_date))
        
        time_left = dt_end_date - dt_next_date
        dt_start_date = dt_next_date + dt.timedelta(days=1)
        
    # Handle the remaining days
    start_date = dt.datetime.strftime(dt_start_date, "%Y-%m-%d")
    dt_next_date = dt_start_date + dt.timedelta(days=time_left.days - 1)
    next_date = dt.datetime.strftime(dt_next_date, "%Y-%m-%d")
    date_intervals.append((start_date, next_date))
        
    page = 1
    
    for start_date, end_date in date_intervals:
        while page <= 10:
            url = f'https://api.github.com/search/repositories?q={keyword} created:{start_date}..{end_date}&per_page={per_page}&page={page}'
            r = requests.get(url)

            if r.ok == True:
                json_pydata = r.json()
                if len(json_pydata['items']) == 0: # no items left on that page
                    page = 1
                    break
                
                all_repositories.extend(json_pydata['items'])    

                page += 1
    
            else:
                time.sleep(1)
    
    return all_repositories

In [12]:
# TEST
start_date = '2017-01-01'
end_date = '2017-01-31'
step = 10
keyword = 'data science'
per_page = 100
all_repositories = collect_all_repositories(start_date, end_date, step, keyword, per_page)
print(len(all_repositories))

url = f'https://api.github.com/search/repositories?q={keyword} created:{start_date}..{end_date}'
finish = False
while finish == False:
    r = requests.get(url)
    if r.ok == True:
        json_pydata = r.json()
        if json_pydata['incomplete_results'] == False: 
            total_count = json_pydata['total_count']
            finish = True
    else:
        time.sleep(1)
assert len(all_repositories) == total_count

1570
